# Energy Reconstruction Using CNN

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Attention

from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

## FIX THIS!
simPrefix = 'D:\icecube\sim_data'

In [2]:
# Name for model
key = 'best_wo_zenith'

# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':'product', 't':False, 'normed':True} #i dont understand exactly what all of these options are

In [3]:
x, y = load_preprocessed(simPrefix, 'train')
print(y.keys())

Percentage of events with a NaN: 2.68
dict_keys(['comp', 'energy', 'dir', 'plane_dir', 'laputop_dir', 'small_dir'])


In [13]:
from tensorflow.keras import Input, layers, Model
import tensorflow as tf
from tensorflow.keras.layers import Concatenate, Dropout, LayerNormalization


inputs = Input(shape=(10,10,1))


o = layers.Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,1))(inputs)
o = layers.Dropout(.2)(o)
o = layers.BatchNormalization(axis=1)(o)
o = layers.Conv2D(32, kernel_size=3, activation='relu')(o)
o = layers.Dropout(.2)(o)
o = layers.BatchNormalization(axis=1)(o)
o = layers.Conv2D(16, kernel_size=3, activation='relu')(o)
o = layers.Dropout(.2)(o)
o = layers.BatchNormalization(axis=1)(o)
o = Flatten()(o)
outputs = Dense(1, activation='relu')(o)
#o = layers.LayerNormalization(axis=1)(o)
#o = layers.Dropout(.3)(o)
#outputs = Dense(1, activation='relu')(o)
#outputs = layers.GlobalAveragePooling2D()(o)

model = Model(inputs=inputs, outputs= outputs ,name="attn_test")


model.summary()

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])


Model: "attn_test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 10, 10, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 64)          640       
_________________________________________________________________
dropout_14 (Dropout)         (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 8, 8, 64)          32        
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 6, 6, 32)          18464     
_________________________________________________________________
dropout_15 (Dropout)         (None, 6, 6, 32)          0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 6, 6, 32)          24

In [10]:
energy = y['energy']
comp = y['comp']
theta, phi = y['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)
temp_y = energy


In [14]:
history = model.fit(x_i[trainCut], temp_y[trainCut], validation_data=(x_i[testCut], temp_y[testCut]), epochs=5, 
                    batch_size = 750)



Epoch 1/5
 83/623 [==>...........................] - ETA: 2:32 - loss: 5.2123 - mse: 5.2123

KeyboardInterrupt: 

In [8]:
# Save model to file for easy loading
## WHERE ARE YOU SAVING TO?
model.save('model_%s.h5' % key)
f = open('model_%s.np' % key, 'wb')
np.save(f, history.history)
